In [ ]:
import tarfile 
import os
import urllib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

### Carregando os dados

In [ ]:
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

In [ ]:
def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, 'housing.tgz')
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()


def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, 'housing.csv')
    return pd.read_csv(csv_path)



In [ ]:
fetch_housing_data()

## Explorando os dados

In [ ]:
housing = load_housing_data()

In [ ]:
housing.head()

In [ ]:
housing.info()

In [ ]:
housing.describe()

In [ ]:
housing['ocean_proximity'].value_counts()

In [ ]:
housing.hist(bins=50, figsize=(20,15))
plt.show()

## Dividindo o conjunto de dados em treino e teste

- Essa parte é essencial de ser feita logo no começo para evitar spoofing bias.

In [ ]:
def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]



In [ ]:
train_set, test_set = split_train_test(housing, 0.2) # A primeira coisa a se fazer depois que garantimos a integridade dos dados é separar o conjunto de teste (20% dos dados) para evitar o data snooping bias.

In [ ]:
print("Conjunto de treino:", len(train_set))
print("Conjunto de teste:", len(test_set))


In [ ]:
# Uma segunda forma mais estável de dividir os dados é usar uma função hash para garantir que a divisão permaneça a mesma mesmo que novos dados sejam adicionados ao conjunto.

from zlib import crc32

def test_set_check(identifier, test_ratio):
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32

def split_train_test_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]    

# Pra aplicar essa função é ideal que tenhamos um ID para coluna, como nosso dataset temos duas alternativas: Criar uma coluna com o indice da linha ou usar a lat/long paara criar um novo id
housing_with_id = housing.reset_index()   # Usando o índice como ID
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "index") # Usando o índice como ID

housing_with_id["id"] = housing["longitude"] * 1000 + housing["latitude"] # Usando a lat/long como ID
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "id") # Usando a lat/long como ID




In [ ]:
# A terceira forma é utilizando as funções do sklearn

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)


In [ ]:
# A quarta forma é fazer uma divisão estratificada, mas que ambos conjuntos representem bem diferentes faixas de renda

# Mas como não temos a categoria de faixa de renda o nosso stakeholder nos ajudou como fazer essa defiinição

housing['Faixa_de_Renda'] = pd.cut(housing['median_income'], bins=[0., 1.5, 3.0, 4.5, 6., np.inf], labels=[1, 2, 3, 4, 5])

housing['Faixa_de_Renda'].hist()

In [ ]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing['Faixa_de_Renda']):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [ ]:
# Vamos validar se a divisão estratificada funcionou comprando as proporções do conjunto total com o conjunto de teste

print(housing['Faixa_de_Renda'].value_counts() / len(housing))


print(strat_test_set['Faixa_de_Renda'].value_counts() / len(strat_test_set))

In [ ]:
# A categoria de faixa de renda realmente ajudou a fazer uma divisão mais representativa do conjunto de dados mas não vamos utilizá-la
# a ideia era usar como uma guia para a amostragem estratificada, nesse caso podemos agora remover essa coluna.

for set_ in (strat_train_set, strat_test_set):
    set_.drop("Faixa_de_Renda", axis=1, inplace=True)   


### Explorando os dados (de treino)

In [ ]:
houseing = strat_train_set.copy()

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4, s=housing['population']/100, label="População", figsize=(10,7), c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True,)
plt.legend()
plt.show()

In [ ]:
housing_coor = housing.iloc[:, :-2].copy()


In [ ]:
coor_matrix = housing_coor.corr()

In [ ]:
coor_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
from pandas.plotting import scatter_matrix

attributes = ["median_house_value", "median_income", "total_rooms", "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12, 8))
plt.show()

In [ ]:
housing.plot(kind='scatter',
             x='median_income',
             y='median_house_value',
             alpha=0.05
             )

In [ ]:
# Vamos criar algumas novas variáveis baseadas em outras para ver se conseguimos melhorar a correlação com o valor da casa
housing['rooms_per_household'] = housing['total_rooms'] / housing['households']
housing['bedrooms_per_room'] = housing['total_bedrooms'] / housing['total_rooms']
housing['population_per_household'] = housing['population'] / housing['households']


In [ ]:
housing.plot(kind="scatter", x="bedrooms_per_room", y="median_house_value", alpha=0.1)

In [ ]:
housing_coor = housing.drop(['Faixa_de_Renda', 'ocean_proximity'], axis=1)

In [ ]:
corr_matrix = housing_coor.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

## Vamos resetar o dataset

In [ ]:
housing_labels = housing["median_house_value"].copy()
housing = housing.drop("median_house_value", axis=1)  # drop labels for training set


In [ ]:
# A maioria dos algoritmos não lida muito bem com dados ausentes, para isso devemos tratálos

# Opção 1 - Deletar as linhas com dados ausentes
# housing.dropna(subset=["total_bedrooms"])    # option 1

# Opção 2 - Deletar a coluna com dados ausentes
# housing.drop("total_bedrooms", axis=1)       # option 2

# Opção 3 - Preencher os dados ausentes com a mediana da coluna
median = housing["total_bedrooms"].median()
housing["total_bedrooms"].fillna(median, inplace=True)


# No caso de inputar devemos garantir que esse valores sejam salvos para que possamos novamente inputas no conjunto de teste.

In [ ]:
# Opção 4 - Usar a biblioteca do sklearn para fazer o preenchimento dos dados ausentes
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median") # Criando o objeto imputer que vai substituir os valores ausentes pela mediana da coluna
# Como a variável categórica "ocean_proximity" não é numérica devemos removê
housing_num = housing.drop("ocean_proximity", axis=1)
imputer.fit(housing_num) # Calculando a mediana de cada coluna

X = imputer.transform(housing_num) # Aplicando o cálculo da mediana para substituir os valores ausentes
housing_tr = pd.DataFrame(X, columns=housing_num.columns, index=housing_num.index)

In [ ]:
housing_tr

### Manipulando rexto e atributos categóricos

In [ ]:
housing_cat = housing[["ocean_proximity"]]
housing_cat.head(10)

In [ ]:
housing['ocean_proximity'].value_counts()

In [ ]:
# A melhor forma de lidar com variáveis categóricas é trocando as classes por números. 
# O scikit-learn tem uma classe para fazer isso chamada OrdinalEncoder ou OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]

In [ ]:
# Para sabermos qual número representa cada categoria
ordinal_encoder.categories_

In [ ]:
## O ORdinal enconder atribui um número para cada categoria, mas o algoritmo pode interpretar que uma categoria é maior que a outra, o que não faz sentido.
## Uma alternativa é usar o OneHotEncoder que cria uma coluna para cada categoria e atribui 0 ou 1 para indicar a presença ou ausência da categoria

cat_enconder = OneHotEncoder()
housing_cat_1hot = cat_enconder.fit_transform(housing_cat)
housing_cat_1hot

In [ ]:
# Observe que o resultado fica em uma matriz espara.
# Issoé útil para economizar espaço, já que o que nós interessa mesmo é a posição dos valors 1. 
# Economia de espaço é muito importante uma vez que em datasets muito grandes o número de colunas pode ser enorme.



### Customize os transformadores

-- Essa parte é importante pois muitaz vezes precisamos criar um pipeline e para isso nos aproveitamos da arquitetura do
scikitlearn para incluir essas transformações diretamente no pipeline.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

### Customize os transformadores

rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True): # Nenhum outro argumento além de self deve ser passado para o init
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # Nada para fazer
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)


### Escaloamento de características

-- Nada mais é do que padronizar ou escolanr uma variável quantitativa. Essa escolha vai depender da antureza dos dados e objetivo do estudo. 
-- No entanto bom lembrar que essas modificações serão feitas apenas no conjunto de dados de treinamento.


### Transformação de pipelines

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")), 
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler())
        ])

housing_num_tr = num_pipeline.fit_transform(housing_num)
housing_num_tr

Como pudemos ver aqui fizemos várias transformações nos dados. Caso isso entre em produção é inviável ter que refazer essas operações toda hora.
Nesse snetido existe uma classe do scikitleanr chamada Pipeline, que faz justamente isso, um pipeline. 

Pra usar nós definimos um nome, chamamos a classe e inserimos dentro dela uma lista contendo tuplas que seguem a seguinte lógica (Nome da etapa, nome do transformador).

A classe Pipeline tem um método chamado fit_transform e toda vez que acionamos ele cada transformador é executado em sequência.



In [ ]:
num_attribs = list(housing_num)
cat_attribs = ['ocean_proximity']

from sklearn.compose import ColumnTransformer

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
        ])



In [ ]:
housing

In [ ]:
# Atualize os atributos numéricos para remover 'median_house_value'
num_attribs = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income']

# Recrie o pipeline completo com os nomes corretos das colunas
full_pipeline = ColumnTransformer([
		("num", num_pipeline, num_attribs),
		("cat", OneHotEncoder(), cat_attribs),
		])

housing_prepared = full_pipeline.fit_transform(housing)

In [ ]:
Unificamos o pipeline numérico com o categórico em um único pipeline

### Escolha e treine o seu modelo

In [ ]:
housing_labels = housing["median_house_value"].copy()

In [ ]:
housing_labels

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

In [ ]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]

some_data_prepared = full_pipeline.transform(some_data)


In [ ]:
some_data_prepared = full_pipeline.fit_transform(some_data)
housing_prepared = full_pipeline.fit_transform(housing)

In [ ]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5] 

some_data_prepared = full_pipeline.transform(some_data)
#housing_prepared = full_pipeline.fit_transform(housing)

#lin_reg.predict(housing_prepared) # It works
lin_reg.predict(some_data_prepared) # It dont

In [ ]:
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)

In [ ]:
lin_rmse

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

In [ ]:
housing_predicitons = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predicitons)

In [ ]:
tree_rmse = np.sqrt(tree_mse)
print(tree_rmse)

## Avaliando melhor com a validação cruzada

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, housing_prepared, housing_labels,scoring="neg_mean_squared_error", cv=10) # Aqui nós usamos o scoring como "neg_mean_squared_error" porque o cross_val_score sempre assume que uma pontuação maior é melhor, então ele inverte o sinal do MSE.
tree_rmse_scores = np.sqrt(-scores) # Aqui é negativo porque o cross_val_score sempre assume que uma pontuação maior é melhor, então ele inverte o sinal do MSE.




In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Média:", scores.mean())
    print("Desvio Padrão:", scores.std())

display_scores(tree_rmse_scores)

In [ ]:
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)


In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor()

#forest_reg.fit(housing_prepared, housing_labels)



forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=2)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)


### Aperfeiçoe o seu modelo

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
 ]
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring="neg_mean_squared_error", return_train_score=True)     
grid_search.fit(housing_prepared, housing_labels)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.cv_results_


for mean_score, params in zip(grid_search.cv_results_["mean_test_score"], grid_search.cv_results_["params"]):
    print(np.sqrt(-mean_score), params)

## Avalie seu sistema noconjunto de testes

In [ ]:
# Certifique-se que strat_test_set está definido
final_model = grid_search.best_estimator_

X_test = strat_test_set.drop('median_house_value', axis=1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

KeyError: 'Faixa_de_Renda'